# Remote Backend Compiler - RBC
*by Pearu Peterson*

## Statement of the problem

*Compile and run user-defined functions in a language-agnostic JIT enabled program semi-remotely.*

### Semi-remote compilation

- *compiler frontend*: user-written source code is parsed and transformed to LLVM IR in client program
- the client program sends the LLVM IR over network to a server program
- *compiler backend*: the server program compiles the LLVM IR to machine code (that can be executed on the server's CPU or GPU devices)

### Constraints

- A user-defined function is defined in Python or C/C++
  (or any other language with LLVM tools)
- Client host runs on a 32- or 64-bit OS: Linux, MacOSX, Windows
- Server host runs on a 64-bit Linux

## Solution

- *Remote Backend Compiler - RBC*: https://github.com/xnd-project/rbc
- A Python-to-Python prototype uses Numba for LLVM IR generation and Numba llvmlite for machine code compilation
- A Python-to-SQL application uses Numba for LLVM IR generation and HeavyDB for machine code compilation
  * HeavyDB is a GPU enabled SQL database server
  * HeavyDB uses LLVM Compiler C++ library for JIT compilation
  * https://www.heavy.ai/

# Demo of the prototype below

In [1]:
#NBVAL_IGNORE_OUTPUT
from rbc import RemoteJIT
rjit = RemoteJIT(host='localhost', port=7890)
rjit.start_server(background=True)

staring rpc.thrift server: /home/pearu/git/xnd-project/rbc/rbc/remotejit.thrift

In [2]:
# Client-server connector, RBC jit-decorator
rjit = RemoteJIT(host='localhost', port=7890)

In [3]:
# A user-defined function
@rjit('int64(int64, int64)')
def foo(a, b):
    return a + b

In [4]:
# NBVAL_IGNORE_OUTPUT
# Generate LLVM IR, useful for debugging
print(foo)


--------------------------------------cpu---------------------------------------
; ModuleID = 'rbc.irtools.compile_to_IR'
source_filename = "<string>"
target triple = "x86_64-unknown-linux-gnu"

@_ZN08NumbaEnv8__main__7foo_241B40c8tJTC_2fWQA9HW1CcAv0EjzIkAdRoAEtoAgA_3dExx = common local_unnamed_addr global i8* null

; Function Attrs: norecurse nounwind readnone
define i64 @foo_lallA(i64 %.1, i64 %.2) local_unnamed_addr #0 {
entry:
  %.6.i = add nsw i64 %.2, %.1
  ret i64 %.6.i
}

attributes #0 = { norecurse nounwind readnone }

!llvm.module.flags = !{!0}

!0 = !{i32 4, !"pass_column_arguments_by_value", i1 false}

--------------------------------------------------------------------------------


In [5]:
# Triggers:
#  - local frontend compile,
#  - remote backend compile,
#  - and remote call
foo(1, 3)

4

In [6]:
foo(2, 4)  # reuses remote compile result and triggers remote call

6

In [7]:
try:
    foo(1.2, 3.4)
except Exception as msg:
    print(msg)

found no matching function signature to given argument types:
    (float64, float64) -> ...
  available function signatures:
    int64(int64, int64)


In [8]:
# update with new signatures
foo.signature('double(double, double)')
foo.signature('complex128(complex128, complex128)');

In [9]:
foo(1, 3.4j)  # triggers remote compile and call

(1+3.4j)

In [10]:
foo(1.2, 3)  # triggers remote compile and call

4.2

In [11]:
rjit.stop_server()